In [5]:
!pip install appdirs==1.4.4
!pip install bitsandbytes==0.37.2
!pip install datasets==2.10.1
!pip install fire==0.5.0
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch==2.0.0
!pip install sentencepiece==0.1.97
!pip install tensorboardX==2.6
!pip install gradio==3.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 5.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 64.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 46.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import transformers
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

# Load the training data
with open("modified_faq_data.json", "r") as f:
    training_data = json.load(f)


/home/mane/anaconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Convert the training data to a format that DialoGPT can understand
training_inputs = []
for item in training_data:
  training_inputs.append(item['question'])
  training_inputs.append(item['answer'])

training_inputs[0:4]

['What is the difference between a Commercial Representative and a Distributor?',
 'A MANE Commercial Representative is appointed to assist for the promotion and sale of MANE’s products (fragrances/ flavours/ ingredients), within a limited territory (country, region, city…) for a specific market (fragrance/ flavour for the food/ cosmetic/ personal care/ home care, and less frequently, in fine fragrance, pharmaceutical, tobacco, pet food, and oral care industries), and grow MANE’s direct sales accordingly. A MANE Distributor is appointed to resell “as is” some of MANE’s products (fragrances/ flavours/ ingredients), within a limited territory (country, region, city…) for a specific market (fragrance/ flavour for the food/ cosmetic/ personal care/ home care, and less frequently, in fine fragrance, pharmaceutical, tobacco, pet food, and oral care industries) to its own customers.',
 'What is the difference between a Commercial Representative and an Agent?',
 'An Agent is entitled to negoti

In [3]:
# Tokenize the training data
tokenizer.pad_token = tokenizer.eos_token
encoded_inputs = tokenizer(training_inputs, truncation=True, padding=True, return_tensors="pt")

# Train the DialoGPT model
model.train()
batch_size = 32

for epoch in range(5):
    for i in range(0, len(training_inputs), batch_size):
        input_ids = encoded_inputs["input_ids"][i:i + batch_size]
        attention_mask = encoded_inputs["attention_mask"][i:i + batch_size]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)

In [4]:
outputs

CausalLMOutputWithCrossAttentions(loss=tensor(6.1899, grad_fn=<NllLossBackward0>), logits=tensor([[[  26.2890,   21.7951,   21.3960,  ...,   24.9814,   23.4212,
            26.0549],
         [ 287.6745,  258.9225,  250.7185,  ...,  270.2687,  284.7589,
           298.0851],
         [ 362.4265,  321.1185,  320.8878,  ...,  353.5522,  357.8240,
           378.1093],
         ...,
         [ 356.5324,  320.6523,  319.4789,  ...,  361.0578,  360.1472,
           373.2766],
         [ 286.6819,  249.0082,  246.4947,  ...,  270.0001,  271.5103,
           286.9873],
         [ 332.3404,  291.6013,  287.7298,  ...,  318.2843,  314.5355,
           333.7718]],

        [[  23.7511,   18.9788,   18.4066,  ...,   22.2116,   20.7400,
            22.8431],
         [ -86.7761,  -85.0092,  -87.9491,  ...,  -90.0054, -101.6889,
           -86.8195],
         [ 280.6041,  263.3658,  262.2711,  ...,  288.2859,  286.8707,
           296.3590],
         ...,
         [ 343.3990,  293.5405,  292.5195, 